In [1]:
import jax.numpy as jnp
import numpy as np

import jax
import urdf_loader

In [7]:
chain = urdf_loader.read_chain_from_urdf('data/kuka_iiwa.urdf', 'lbr_iiwa_link_0', 'lbr_iiwa_link_7')
kinematics = urdf_loader.make_kinematic_chain_function(chain)
kinematics_j = jax.jit(kinematics)
zero_pose = jnp.array([0., 0., 0., 0., 0., 0., 0.])
kinematics(zero_pose)

DeviceArray([6.0889960e-08, 1.4274991e-01, 9.5369828e-01, 9.9999988e-01,
             0.0000000e+00, 4.3711388e-08, 1.3113416e-07], dtype=float32)

In [10]:
joint_configurations = np.random.rand(100, 7)
ans = jax.vmap(kinematics_j)(joint_configurations)

In [11]:
%timeit jax.vmap(kinematics_j)(joint_configurations)

737 µs ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit jax.vmap(kinematics)(joint_configurations)

985 ms ± 56.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit kinematics_j(zero_pose)

The problem that I nominally want to solve is:

$$
\min_{x(t)} \int_0^T L\left(x(t), u(t)\right)dt + \Phi\left(x(T)\right) \\
s.t. \\
\dot{x}(t) = f\left(x(t), u(t)\right)\\
x_l < x(t) < x_u
$$

We can discretize the problem with $h=t_{k+1} - t_k$:

$$
\min_{\hat{x}, \hat{u}} \sum_k^{N-1} \frac{h}{2}\left(L_{k+1} + L_k\right) + \Phi\left(x_N\right)
 \\
s.t.\\
x_c = \left(\frac{1}{2}\left(x_{k+1} + x_k\right) - \frac{h}{8}\left(f_{k+1} - f_k\right)\right) \\
0 = \frac{3}{2h}\left(\left(x_{k+1} - x_k\right) - \frac{h}{6}\left(f_{k+1} + 4f_c + f_k\right) \right) \\
x_l < x_k < x_u
$$



Including the equality constraint gives the following problem:

$$
\min_{\hat{x}, \hat{u}, \hat{\lambda}} \sum_k^{N-1} \frac{h}{2}\left(L_{k+1} + L_k\right) + \sum_k^{N-1} \lambda_k \left(\left(x_{k+1} - x_k\right) - \frac{h}{6}\left(f_{k+1} + 4f_c + f_k\right) \right)  + \Phi\left(x_N\right)
 \\
s.t.\\
x_l < x_k < x_u
$$

The first-order conditions are then:
$$
\frac{\partial V}{\partial x_0} = 
\frac{h}{2}\frac{\partial L_0}{\partial x_0} + 
\lambda_0 \left(-1 - \frac{h}{6}\left(4\frac{\partial f_c}{\partial x_0} + \frac{\partial f_0}{\partial x_0}\right)\right)
$$

$$
\frac{\partial V}{\partial x_{k+1}} = 
\frac{h}{2}\frac{\partial L_{k+1}}{\partial x_{k+1}} + 
\lambda_{k+1} \left(-1 - \frac{h}{6}\left(4\frac{\partial f_{c+1}}{\partial x_{k+1}}+\frac{\partial f_{k+1}}{\partial x_{k+1}}\right)\right)
+
\frac{h}{2}\frac{\partial L_{k+1}}{\partial x_{k+1}} + 
\lambda_k \left(1 - \frac{h}{6}\left(\frac{\partial f_{k+1}}{\partial x_{k+1}} + 4\frac{\partial f_c}{\partial x_{k+1}}\right)\right)
$$

$$
\frac{\partial V}{\partial x_N} = \frac{\partial \Phi}{\partial x_N}
$$

$$
\frac{\partial V}{\partial u_0} =  
\frac{h}{2}\frac{\partial L_0}{\partial u_0} + 
\lambda_0 \left(- \frac{h}{6}\left(4\frac{\partial f_c}{\partial u_0} + \frac{\partial f_0}{\partial u_0}\right)\right)
$$

$$
\frac{\partial V}{\partial u_{k+1}} = 
\frac{h}{2}\frac{\partial L_{k+1}}{\partial u_{k+1}} + 
\lambda_{k+1} \left(- \frac{h}{6}\left(4\frac{\partial f_{c+1}}{\partial u_{k+1}}+\frac{\partial f_{k+1}}{\partial u_{k+1}}\right)\right)
+
\frac{h}{2}\frac{\partial L_{k+1}}{\partial u_{k+1}} + 
\lambda_k \left(- \frac{h}{6}\left(\frac{\partial f_{k+1}}{\partial u_{k+1}} + 4\frac{\partial f_c}{\partial u_{k+1}}\right)\right)
$$

$$
\frac{\partial V}{\partial \lambda_k} = \left(x_{k+1} - x_k\right) - \frac{h}{6}\left(f_{k+1} + 4f_c + f_k\right)
$$

In [ ]:
Plan is to just take the hamiltonian
and discretize it temporaly, then use the magic of 